In [23]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/competitions/cpe-ai-hackathon-2025-medical-data-classification")

Skipping, found downloaded files in "./cpe-ai-hackathon-2025-medical-data-classification" (use force=True to force download)


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [25]:
train = pd.read_csv("/content/cpe-ai-hackathon-2025-medical-data-classification/train.csv")
test = pd.read_csv("/content/cpe-ai-hackathon-2025-medical-data-classification/test.csv")

In [26]:
train.head()

,id,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,Class
0,0,0.292428,0.293124,0.314311,0.358699,0.331543,0.225766,0.239241,0.272733,0.286617,0.288724,0.396327,0.369219,0.454737,0.401026,NaN,0.373793,0
1,1,0.227376,0.371844,0.281638,0.296169,0.165798,0.183871,0.279604,0.508437,0.417472,0.404794,0.117159,0.366740,0.339934,0.513000,0.451442,0.463053,1
2,2,0.630380,0.355120,0.373488,0.266282,0.339572,0.080644,0.164020,0.225593,0.152664,0.250015,0.744645,0.347192,0.353192,0.359334,0.408117,0.447714,1
3,3,0.892770,0.385269,NaN,NaN,0.339572,0.338680,0.325313,0.548815,0.264808,0.395856,0.239452,0.410747,0.452542,0.403616,0.457993,0.457478,1
4,4,0.430202,0.107202,0.463268,0.114090,0.235297,0.167705,0.260785,0.239052,0.174473,0.193488,0.275656,0.173596,0.278163,0.153616,0.286076,0.225951,0


In [27]:
train = train.drop(columns='id')
id_test = test.id
test = test.drop(columns='id')
test = test.drop(columns='Class')

In [28]:
train = train.fillna(train.median())
test = test.fillna(test.median())

x_train = train.drop(columns=['Class'])
y_train = train.Class

In [29]:
x_train = x_train.values.astype(np.float32)
tensor_x_train = torch.tensor(x_train)
y_train = y_train.values.astype(np.float32).reshape(-1, 1)
tensor_y_train = torch.tensor(y_train)

test = test.values.astype(np.float32)
tensor_test = torch.tensor(test)

In [30]:
class neural_network(nn.Module):
    def __init__(self, input_dim):
        super(neural_network, self).__init__()
        self.feature = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.25),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.25),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.25),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.25)
        )
        self.input_bn = nn.BatchNorm1d(input_dim)
        self.output_layer = nn.Linear(32, 1)

    def forward(self, x):
        x = self.input_bn(x)
        x = self.feature(x)
        out = self.output_layer(x)
        return out


In [31]:
model = neural_network(tensor_x_train.shape[1])
pos_weight = torch.tensor([3.0])
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

col_F1, col_F1_val = [], []
epoch, n = 2000, 100

for i in range(0, epoch):
  model.train()
  y_hat = model(tensor_x_train)
  loss_BCE = error(y_hat, tensor_y_train)

  optimizer.zero_grad()
  loss_BCE.backward()
  optimizer.step()

  with torch.no_grad():
    y_hat = (y_hat >= 0.5).float()

  if i == (n-1):
    print(f"Epoch [{i+1}/{epoch}] | Loss(BCE): {loss_BCE:.4f}")
    n += 100


Epoch [100/2000] | Loss(BCE): 0.2385
Epoch [200/2000] | Loss(BCE): 0.1602
Epoch [300/2000] | Loss(BCE): 0.1133
Epoch [400/2000] | Loss(BCE): 0.1104
Epoch [500/2000] | Loss(BCE): 0.1039
Epoch [600/2000] | Loss(BCE): 0.0822
Epoch [700/2000] | Loss(BCE): 0.0760
Epoch [800/2000] | Loss(BCE): 0.0671
Epoch [900/2000] | Loss(BCE): 0.0674
Epoch [1000/2000] | Loss(BCE): 0.0597
Epoch [1100/2000] | Loss(BCE): 0.0663
Epoch [1200/2000] | Loss(BCE): 0.0688
Epoch [1300/2000] | Loss(BCE): 0.0620
Epoch [1400/2000] | Loss(BCE): 0.0540
Epoch [1500/2000] | Loss(BCE): 0.0736
Epoch [1600/2000] | Loss(BCE): 0.0606
Epoch [1700/2000] | Loss(BCE): 0.0553
Epoch [1800/2000] | Loss(BCE): 0.0544
Epoch [1900/2000] | Loss(BCE): 0.0494
Epoch [2000/2000] | Loss(BCE): 0.0468


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
with torch.no_grad():
  y_hat = model(tensor_test)
  y_hat = (y_hat > 0.5).float()
  y_hat = y_hat.squeeze().numpy().astype(int)
y_hat = pd.DataFrame(y_hat)

summarize = pd.concat([id_test, y_hat], axis=1)
summarize.columns = ['id', 'Class']
summarize.to_csv('My_Answer_data.csv', index=False)
print(summarize['Class'].unique())
summarize


[0 1]


,id,Class
0,0,0
1,1,0
2,2,0
3,3,1
4,4,1
...,...,...
7483,7483,1
7484,7484,0
7485,7485,0
7486,7486,1
